# Explore 2025 Stocking Data

This notebook explores the new 2025 stocking data and identifies:
- Bodies of water that are new (not in existing 2024 data)
- Bodies of water that are missing coordinates
- Differences between 2024 and 2025 data

**Goal:** Prepare for updating the map with 2025 data while preserving pike and char data.


In [1]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path

print("Libraries imported successfully!")


Libraries imported successfully!


In [2]:
# Load 2025 stocking data
df_2025 = pd.read_excel('2025_Stocking.xlsx')

print(f"2025 Data Shape: {df_2025.shape}")
print(f"\nColumns: {df_2025.columns.tolist()}")
print(f"\nFirst few rows:")
df_2025.head()

# Show data types and sample values
print(f"\nData types:")
print(df_2025.dtypes)
print(f"\nSample values from each column:")
for col in df_2025.columns:
    print(f"\n{col}: {df_2025[col].unique()[:10]}")


2025 Data Shape: (2207, 21)

Columns: ['Id', 'WATCODE', 'WATER', 'TOWN', 'COUNTY', 'REGION', 'SPP', 'STRAIN', 'AGEGRP', 'QTYRequested', 'QTYDelivered', 'KGs', 'mm', 'Inches', 'MARKS', 'DeliveryDate', 'HATCHERY', 'ORDER TYPE', 'SITE', 'COMMENTS', 'NOTE']

First few rows:

Data types:
Id                       int64
WATCODE                 object
WATER                   object
TOWN                    object
COUNTY                  object
REGION                  object
SPP                     object
STRAIN                   int64
AGEGRP                  object
QTYRequested             int64
QTYDelivered             int64
KGs                    float64
mm                       int64
Inches                   int64
MARKS                   object
DeliveryDate    datetime64[ns]
HATCHERY                object
ORDER TYPE              object
SITE                    object
COMMENTS                object
NOTE                    object
dtype: object

Sample values from each column:

Id: [46266 50951 

/Users/jonathan/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
# Load existing combined data
df_current = pd.read_csv('df_updated.csv')

print(f"Current Data Shape: {df_current.shape}")
print(f"\nColumns: {df_current.columns.tolist()}")
print(f"\nUnique species in current data: {sorted(df_current['SPECIES'].unique())}")
print(f"\nFirst few rows:")
df_current.head()


Current Data Shape: (2250, 10)

Columns: ['COUNTY', 'DATE', 'WATER', 'TOWN', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord', 'ABUNDANCE']

Unique species in current data: ['ARCTIC CHAR', 'BROOK TROUT', 'BROWN TROUT', 'L.L. SALMON', 'LAKE TROUT', 'NORTHERN PIKE', 'RAINBOW TROUT', 'SPLAKE']

First few rows:


,COUNTY,DATE,WATER,TOWN,SPECIES,QTY,SIZE (inch),X_coord,Y_coord,ABUNDANCE
0,Androscoggin,2025-02-11,RANGE P (LOWER),Poland,BROOK TROUT,300.0,14.0,-70.369148,44.043731,NaN
1,Cumberland,2025-04-03,SONGO R,Casco,BROOK TROUT,100.0,16.0,-70.574503,43.930731,NaN
2,Cumberland,2025-04-03,JORDAN R,Raymond,BROOK TROUT,100.0,16.0,-70.462052,43.896887,NaN
3,Cumberland,2025-04-03,PRESUMPSCOT R,Windham,BROOK TROUT,200.0,16.0,-70.410724,43.717251,NaN
4,Cumberland,2025-04-07,COLLYER B,Gray,BROOK TROUT,100.0,10.0,-70.302713,43.905907,NaN


In [4]:
# Standardize 2025 data - we need: TOWN, DATE, WATER, COUNTY, SPECIES, QTY, SIZE (inch)
# First normalize column names to uppercase
df_2025.columns = df_2025.columns.str.strip().str.upper()

# Map actual column names to standard names based on what we see in the data
# From the output: DELIVERYDATE -> DATE, INCHES -> SIZE (inch), QTYDELIVERED -> QTY, SPP -> SPECIES
column_mapping = {
    'DELIVERYDATE': 'DATE',
    'DATE': 'DATE',  # Keep if already named correctly
    'INCHES': 'SIZE (inch)',
    'QTYDELIVERED': 'QTY',  # Use delivered quantity
    'QTYREQUESTED': 'QTY',  # Fallback to requested if delivered not available
    'QUANTITY': 'QTY',
    'SPP': 'SPECIES',  # Species code column
    'SPECIES': 'SPECIES'  # Keep if already named correctly
}

# Rename columns based on mapping (prioritize first match)
for old_name, new_name in column_mapping.items():
    if old_name in df_2025.columns:
        if new_name not in df_2025.columns or old_name == new_name:
            df_2025 = df_2025.rename(columns={old_name: new_name})

# Handle QTY - use QTYDELIVERED if available, otherwise QTYREQUESTED
if 'QTYDELIVERED' in df_2025.columns and 'QTY' not in df_2025.columns:
    df_2025['QTY'] = df_2025['QTYDELIVERED']
elif 'QTYREQUESTED' in df_2025.columns and 'QTY' not in df_2025.columns:
    df_2025['QTY'] = df_2025['QTYREQUESTED']

# Map species codes to full species names
# BKT = brook trout, RBT = rainbow trout, LKT = lake trout, 
# BNT = brown trout, LLS = landlocked salmon, SPK = splake
species_code_mapping = {
    'BKT': 'BROOK TROUT',
    'RBT': 'RAINBOW TROUT',
    'LKT': 'LAKE TROUT',
    'BNT': 'BROWN TROUT',
    'LLS': 'L.L. SALMON',
    'SPK': 'SPLAKE'
}

# Convert species codes to full names
if 'SPECIES' in df_2025.columns:
    df_2025['SPECIES'] = df_2025['SPECIES'].str.strip().str.upper().map(
        lambda x: species_code_mapping.get(x, x) if pd.notna(x) else x
    )

# Select only the columns we need: TOWN, DATE, WATER, COUNTY, SPECIES, QTY, SIZE (inch)
required_columns = ['TOWN', 'DATE', 'WATER', 'COUNTY', 'SPECIES', 'QTY', 'SIZE (inch)']
available_columns = [col for col in required_columns if col in df_2025.columns]

# Keep only required columns
df_2025_clean = df_2025[available_columns].copy()

# Add missing columns as None if they don't exist
for col in required_columns:
    if col not in df_2025_clean.columns:
        df_2025_clean[col] = None
        print(f"Warning: Column '{col}' not found in 2025 data - added as None")

# Reorder columns to match required order
df_2025_clean = df_2025_clean[required_columns]

print("Standardized columns in 2025 data:")
print(df_2025_clean.columns.tolist())
print(f"\nSpecies codes found and mapped:")
print(f"Unique species values: {sorted(df_2025_clean['SPECIES'].dropna().unique())}")
print(f"\nData shape: {df_2025_clean.shape}")
print(f"\nFirst few rows after standardization:")
df_2025_clean.head()

# Use the clean dataframe going forward
df_2025 = df_2025_clean.copy()


Standardized columns in 2025 data:
['TOWN', 'DATE', 'WATER', 'COUNTY', 'SPECIES', 'QTY', 'SIZE (inch)']

Species codes found and mapped:
Unique species values: ['BROOK TROUT', 'BROWN TROUT', 'L.L. SALMON', 'LAKE TROUT', 'RAINBOW TROUT', 'SPLAKE']

Data shape: (2207, 7)

First few rows after standardization:


In [5]:
# Extract only stocked species from current data (exclude pike and char for now)
# These are permanent species that we'll keep
permanent_species = ['NORTHERN PIKE', 'ARCTIC CHAR']

# Get current stocked species data (trout, salmon, splake - these will be replaced)
current_stocked = df_current[~df_current['SPECIES'].isin(permanent_species)].copy()

print(f"Current stocked species data rows: {len(current_stocked)}")
print(f"\nSpecies in current stocked data: {sorted(current_stocked['SPECIES'].unique())}")

# Get permanent species data (pike and char - these we'll keep)
permanent_data = df_current[df_current['SPECIES'].isin(permanent_species)].copy()
print(f"\nPermanent species data rows: {len(permanent_data)}")
print(f"Species in permanent data: {sorted(permanent_data['SPECIES'].unique())}")


Current stocked species data rows: 2207

Species in current stocked data: ['BROOK TROUT', 'BROWN TROUT', 'L.L. SALMON', 'LAKE TROUT', 'RAINBOW TROUT', 'SPLAKE']

Permanent species data rows: 43
Species in permanent data: ['ARCTIC CHAR', 'NORTHERN PIKE']


In [6]:
# Create a coordinate lookup from current data
# Key: (WATER, TOWN) tuple, Value: (X_coord, Y_coord) tuple
coord_lookup = {}

for _, row in df_current.iterrows():
    if pd.notna(row['X_coord']) and pd.notna(row['Y_coord']):
        key = (str(row['WATER']).strip().upper(), str(row['TOWN']).strip().upper())
        coord_lookup[key] = (float(row['X_coord']), float(row['Y_coord']))

print(f"Found coordinates for {len(coord_lookup)} unique waterbody/town combinations")
print(f"\nSample of coordinate lookup (first 5):")
for i, (key, coords) in enumerate(list(coord_lookup.items())[:5]):
    print(f"  {key[0]}, {key[1]}: ({coords[0]:.6f}, {coords[1]:.6f})")


Found coordinates for 943 unique waterbody/town combinations

Sample of coordinate lookup (first 5):
  RANGE P (LOWER), POLAND: (-70.369148, 44.043731)
  SONGO R, CASCO: (-70.574503, 43.930731)
  JORDAN R, RAYMOND: (-70.462052, 43.896887)
  PRESUMPSCOT R, WINDHAM: (-70.410724, 43.717251)
  COLLYER B, GRAY: (-70.302713, 43.905907)


In [7]:
# Prepare 2025 data for analysis
# Add empty coordinate columns (will be filled from existing data or left blank)
df_2025_prep = df_2025.copy()
df_2025_prep['X_coord'] = None
df_2025_prep['Y_coord'] = None

# Create standardized keys for matching (WATER, TOWN)
df_2025_prep['key'] = df_2025_prep.apply(
    lambda row: (str(row['WATER']).strip().upper(), str(row['TOWN']).strip().upper()), 
    axis=1
)

# Check unique waterbodies in 2025 data
unique_2025_locations = df_2025_prep[['WATER', 'TOWN', 'COUNTY']].drop_duplicates()
print(f"Unique waterbody/town combinations in 2025 data: {len(unique_2025_locations)}")
print(f"\nTotal rows in 2025 data: {len(df_2025_prep)}")
print(f"\nSpecies in 2025 data: {sorted(df_2025_prep['SPECIES'].dropna().unique())}")
print(f"\nColumns in 2025 data: {df_2025_prep.columns.tolist()}")


Unique waterbody/town combinations in 2025 data: 921

Total rows in 2025 data: 2207

Species in 2025 data: ['BROOK TROUT', 'BROWN TROUT', 'L.L. SALMON', 'LAKE TROUT', 'RAINBOW TROUT', 'SPLAKE']

Columns in 2025 data: ['TOWN', 'DATE', 'WATER', 'COUNTY', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord', 'key']


In [8]:
# Identify which 2025 waterbodies have coordinates and which don't
df_2025_prep['has_coords'] = df_2025_prep['key'].apply(
    lambda k: k in coord_lookup
)

# For those with coordinates, populate them
for idx, row in df_2025_prep.iterrows():
    if row['has_coords']:
        coords = coord_lookup[row['key']]
        df_2025_prep.at[idx, 'X_coord'] = coords[0]
        df_2025_prep.at[idx, 'Y_coord'] = coords[1]

# Get unique locations that are missing coordinates
missing_coords = df_2025_prep[~df_2025_prep['has_coords']][['WATER', 'TOWN', 'COUNTY', 'SPECIES']].drop_duplicates()

print(f"Waterbodies WITH coordinates: {df_2025_prep['has_coords'].sum()} rows")
print(f"Waterbodies MISSING coordinates: {(~df_2025_prep['has_coords']).sum()} rows")
print(f"\nUnique waterbody/town combinations MISSING coordinates: {len(missing_coords)}")
print(f"\nUnique waterbody/town combinations WITH coordinates: {df_2025_prep[df_2025_prep['has_coords']][['WATER', 'TOWN']].drop_duplicates().shape[0]}")


Waterbodies WITH coordinates: 2189 rows
Waterbodies MISSING coordinates: 18 rows

Unique waterbody/town combinations MISSING coordinates: 14

Unique waterbody/town combinations WITH coordinates: 908


In [9]:
# Show locations missing coordinates (sorted by county, town, water)
print("=" * 80)
print("LOCATIONS MISSING COORDINATES")
print("=" * 80)
print(f"\nTotal unique locations missing coordinates: {len(missing_coords)}\n")

missing_coords_sorted = missing_coords.sort_values(['COUNTY', 'TOWN', 'WATER'])
print(missing_coords_sorted.to_string(index=False))


LOCATIONS MISSING COORDINATES

Total unique locations missing coordinates: 14

                         WATER                  TOWN     COUNTY     SPECIES
            MUSQUACOOK L (2ND)          T11 R11 WELS  Aroostook  LAKE TROUT
                    UMSASKIS L          T11 R13 WELS  Aroostook  LAKE TROUT
               NICATOUS STREAM                 T3 ND    Hancock BROOK TROUT
                       BASIN P               Fayette   Kennebec BROOK TROUT
                       BASIN P               Fayette   Kennebec      SPLAKE
                      DENNIS B            Litchfield   Kennebec BROOK TROUT
                     POTTERS B            Litchfield   Kennebec BROOK TROUT
                       FRESH P           North Haven       Knox BROOK TROUT
BANGOR MUNICIPAL GOLF COURSE P                Bangor  Penobscot BROOK TROUT
                       MOORE P        Bradstreet Twp   Somerset BROOK TROUT
                     KNIGHTS P        Squaretown Twp   Somerset BROOK TROUT
         

In [10]:
# Compare with 2024 data to identify NEW waterbodies
# Get unique locations from 2024 stocked data
unique_2024_locations = current_stocked[['WATER', 'TOWN', 'COUNTY']].drop_duplicates()
unique_2024_locations['key'] = unique_2024_locations.apply(
    lambda row: (str(row['WATER']).strip().upper(), str(row['TOWN']).strip().upper()),
    axis=1
)

# Get unique locations from 2025 data
unique_2025_locations_clean = df_2025_prep[['WATER', 'TOWN', 'COUNTY']].drop_duplicates()
unique_2025_locations_clean['key'] = unique_2025_locations_clean.apply(
    lambda row: (str(row['WATER']).strip().upper(), str(row['TOWN']).strip().upper()),
    axis=1
)

# Find new locations in 2025
set_2024 = set(unique_2024_locations['key'].tolist())
set_2025 = set(unique_2025_locations_clean['key'].tolist())

new_locations = set_2025 - set_2024
removed_locations = set_2024 - set_2025

print("=" * 80)
print("COMPARISON: 2024 vs 2025")
print("=" * 80)
print(f"\nUnique locations in 2024 data: {len(set_2024)}")
print(f"Unique locations in 2025 data: {len(set_2025)}")
print(f"\nNEW locations in 2025 (not in 2024): {len(new_locations)}")
print(f"REMOVED locations from 2024 (not in 2025): {len(removed_locations)}")
print(f"Locations in BOTH: {len(set_2024 & set_2025)}")


COMPARISON: 2024 vs 2025

Unique locations in 2024 data: 921
Unique locations in 2025 data: 921

NEW locations in 2025 (not in 2024): 0
REMOVED locations from 2024 (not in 2025): 0
Locations in BOTH: 921


In [11]:
# Show new locations in 2025
if len(new_locations) > 0:
    print("\n" + "=" * 80)
    print("NEW LOCATIONS IN 2025 (not present in 2024)")
    print("=" * 80)
    new_locations_df = unique_2025_locations_clean[
        unique_2025_locations_clean['key'].isin(new_locations)
    ][['WATER', 'TOWN', 'COUNTY']].sort_values(['COUNTY', 'TOWN', 'WATER'])
    print(f"\nTotal new locations: {len(new_locations_df)}\n")
    print(new_locations_df.to_string(index=False))
else:
    print("\nNo new locations found in 2025 data.")



No new locations found in 2025 data.


In [12]:
# Show removed locations (in 2024 but not in 2025)
if len(removed_locations) > 0:
    print("\n" + "=" * 80)
    print("LOCATIONS REMOVED FROM 2024 (not present in 2025)")
    print("=" * 80)
    removed_locations_df = unique_2024_locations[
        unique_2024_locations['key'].isin(removed_locations)
    ][['WATER', 'TOWN', 'COUNTY']].sort_values(['COUNTY', 'TOWN', 'WATER'])
    print(f"\nTotal removed locations: {len(removed_locations_df)}\n")
    print(removed_locations_df.to_string(index=False))
else:
    print("\nNo locations were removed (all 2024 locations appear in 2025).")



No locations were removed (all 2024 locations appear in 2025).


In [13]:
# Create a summary DataFrame of locations missing coordinates
# This will be useful for manually adding coordinates later
missing_coords_summary = df_2025_prep[~df_2025_prep['has_coords']][
    ['WATER', 'TOWN', 'COUNTY']
].drop_duplicates().sort_values(['COUNTY', 'TOWN', 'WATER'])

# Add columns for manually entering coordinates (initially blank)
missing_coords_summary['X_coord'] = None
missing_coords_summary['Y_coord'] = None

print("=" * 80)
print("SUMMARY: Locations Missing Coordinates")
print("=" * 80)
print(f"\nTotal unique locations missing coordinates: {len(missing_coords_summary)}")
if len(missing_coords_summary) > 0:
    print(f"\nBreakdown by county:")
    print(missing_coords_summary['COUNTY'].value_counts())
    print(f"\nFirst 20 locations (full list below):")
    print(missing_coords_summary.head(20).to_string(index=False))
else:
    print("\n✅ All locations have coordinates!")


SUMMARY: Locations Missing Coordinates

Total unique locations missing coordinates: 13

Breakdown by county:
COUNTY
Kennebec      3
Washington    3
Aroostook     2
Somerset      2
Hancock       1
Knox          1
Penobscot     1
Name: count, dtype: int64

First 20 locations (full list below):
                         WATER                  TOWN     COUNTY X_coord Y_coord
            MUSQUACOOK L (2ND)          T11 R11 WELS  Aroostook    None    None
                    UMSASKIS L          T11 R13 WELS  Aroostook    None    None
               NICATOUS STREAM                 T3 ND    Hancock    None    None
                       BASIN P               Fayette   Kennebec    None    None
                      DENNIS B            Litchfield   Kennebec    None    None
                     POTTERS B            Litchfield   Kennebec    None    None
                       FRESH P           North Haven       Knox    None    None
BANGOR MUNICIPAL GOLF COURSE P                Bangor  Penobscot    

In [ ]:
# Save missing coordinates to CSV for manual coordinate entry
# NOTE: Only run this ONCE before manually adding coordinates!
# Commented out to prevent overwriting manually entered coordinates
# missing_coords_summary.to_csv('missing_coordinates_2025.csv', index=False)
# print("✅ Saved missing coordinates to 'missing_coordinates_2025.csv'")
print("⚠️  Skipping save to prevent overwriting manually entered coordinates")
print(f"Current missing_coordinates_2025.csv has {len(missing_coords_summary)} locations")
print(f"\nTo regenerate the file, uncomment the lines above.")


✅ Saved missing coordinates to 'missing_coordinates_2025.csv'

This file can be used to manually add coordinates.
After adding coordinates, you can use this file to update the main dataset.


## Next Steps

1. **Review the missing coordinates list** - Check `missing_coordinates_2025.csv`
2. **Manually add coordinates** - Fill in X_coord and Y_coord columns for new locations
3. **Create update script** - Build a script that:
   - Loads 2025 data
   - Merges in coordinates (from existing data + manually added)
   - Combines with permanent species data (pike and char)
   - Saves as updated `df_updated.csv`
4. **Update the map** - The map will automatically use the new data

**Note:** The lakefish survey data (`LakeFish_Maine.xls`) is available but will be handled separately in the future.


In [15]:
# Quick stats summary
print("=" * 80)
print("DATA SUMMARY")
print("=" * 80)
print(f"\n2025 Stocking Data:")
print(f"  Total rows: {len(df_2025_prep)}")
print(f"  Unique locations: {len(unique_2025_locations_clean)}")
print(f"  Species: {', '.join(sorted(df_2025_prep['SPECIES'].dropna().unique()))}")
print(f"  Columns: {', '.join(df_2025_prep.columns.tolist())}")

print(f"\nCurrent Combined Data:")
print(f"  Total rows: {len(df_current)}")
print(f"  Stocked species rows: {len(current_stocked)}")
print(f"  Permanent species rows (to keep): {len(permanent_data)}")

print(f"\nCoordinate Status:")
print(f"  Locations with coordinates: {df_2025_prep['has_coords'].sum()} rows")
print(f"  Locations missing coordinates: {(~df_2025_prep['has_coords']).sum()} rows")
print(f"  Unique locations missing coordinates: {len(missing_coords_summary)}")

print(f"\nComparison (2024 vs 2025):")
print(f"  New locations in 2025: {len(new_locations)}")
print(f"  Removed locations from 2024: {len(removed_locations)}")
print(f"  Locations in both: {len(set_2024 & set_2025)}")


DATA SUMMARY

2025 Stocking Data:
  Total rows: 2207
  Unique locations: 921
  Species: BROOK TROUT, BROWN TROUT, L.L. SALMON, LAKE TROUT, RAINBOW TROUT, SPLAKE
  Columns: TOWN, DATE, WATER, COUNTY, SPECIES, QTY, SIZE (inch), X_coord, Y_coord, key, has_coords

Current Combined Data:
  Total rows: 2250
  Stocked species rows: 2207
  Permanent species rows (to keep): 43

Coordinate Status:
  Locations with coordinates: 2189 rows
  Locations missing coordinates: 18 rows
  Unique locations missing coordinates: 13

Comparison (2024 vs 2025):
  New locations in 2025: 0
  Removed locations from 2024: 0
  Locations in both: 921


## Update Data with 2025 Stocking Data

Run the cells below to:
1. Load 2025 data with updated coordinates
2. Merge coordinates from existing data and manually added coordinates
3. Combine with permanent species (pike and char)
4. Update df_updated.csv
5. Create coordinate_reference.csv for future use


In [28]:
# Load manually added coordinates from missing_coordinates_2025.csv
manual_coords = {}
df_manual = pd.read_csv('missing_coordinates_2025.csv')
for _, row in df_manual.iterrows():
    if pd.notna(row['X_coord']) and pd.notna(row['Y_coord']):
        key = (str(row['WATER']).strip().upper(), str(row['TOWN']).strip().upper())
        manual_coords[key] = (float(row['X_coord']), float(row['Y_coord']))

print(f"Loaded {len(manual_coords)} manually added coordinates")


Loaded 13 manually added coordinates


In [29]:
# Merge manual coordinates into 2025 data (prioritize manual over existing)
for idx, row in df_2025_prep.iterrows():
    key = (str(row['WATER']).strip().upper(), str(row['TOWN']).strip().upper())
    
    # Prioritize manual coordinates, then existing
    if key in manual_coords:
        df_2025_prep.at[idx, 'X_coord'] = manual_coords[key][0]
        df_2025_prep.at[idx, 'Y_coord'] = manual_coords[key][1]
    elif key in coord_lookup:
        df_2025_prep.at[idx, 'X_coord'] = coord_lookup[key][0]
        df_2025_prep.at[idx, 'Y_coord'] = coord_lookup[key][1]

# Check coordinate status after merge
has_coords = df_2025_prep['X_coord'].notna() & df_2025_prep['Y_coord'].notna()
print(f"Rows with coordinates: {has_coords.sum()}")
print(f"Rows missing coordinates: {(~has_coords).sum()}")

if (~has_coords).sum() > 0:
    missing = df_2025_prep[~has_coords][['WATER', 'TOWN', 'COUNTY']].drop_duplicates()
    print(f"\n⚠️  Warning: {len(missing)} unique locations still missing coordinates")
    print(missing.head(10).to_string(index=False))
else:
    print("\n✅ All locations have coordinates!")


Rows with coordinates: 2207
Rows missing coordinates: 0

✅ All locations have coordinates!


In [30]:
# Prepare 2025 data for final output - select only required columns
# Remove temporary columns (key, has_coords)
df_2025_final = df_2025_prep[['COUNTY', 'DATE', 'WATER', 'TOWN', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord']].copy()

# Convert QTY and SIZE (inch) to integers using Int64 (nullable integer) dtype
# This allows integers with NaN values and saves as integers in CSV
df_2025_final['QTY'] = df_2025_final['QTY'].apply(
    lambda x: int(float(x)) if pd.notna(x) and str(x).strip() != '' else pd.NA
).astype('Int64')

df_2025_final['SIZE (inch)'] = df_2025_final['SIZE (inch)'].apply(
    lambda x: int(float(x)) if pd.notna(x) and str(x).strip() != '' else pd.NA
).astype('Int64')

# Ensure proper column order
df_2025_final = df_2025_final[['COUNTY', 'DATE', 'WATER', 'TOWN', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord']]

print(f"2025 data ready: {len(df_2025_final)} rows")
print(f"Columns: {df_2025_final.columns.tolist()}")
print(f"Data types - QTY: {df_2025_final['QTY'].dtype}, SIZE (inch): {df_2025_final['SIZE (inch)'].dtype}")
df_2025_final.head()


2025 data ready: 2207 rows
Columns: ['COUNTY', 'DATE', 'WATER', 'TOWN', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord']
Data types - QTY: Int64, SIZE (inch): Int64


,COUNTY,DATE,WATER,TOWN,SPECIES,QTY,SIZE (inch),X_coord,Y_coord
0,Androscoggin,2025-02-11,RANGE P (LOWER),Poland,BROOK TROUT,300,14,-70.369148,44.043731
1,Cumberland,2025-04-03,SONGO R,Casco,BROOK TROUT,100,16,-70.574503,43.930731
2,Cumberland,2025-04-03,JORDAN R,Raymond,BROOK TROUT,100,16,-70.462052,43.896887
3,Cumberland,2025-04-03,PRESUMPSCOT R,Windham,BROOK TROUT,200,16,-70.410724,43.717251
4,Cumberland,2025-04-07,COLLYER B,Gray,BROOK TROUT,100,10,-70.302713,43.905907


In [31]:
# Load permanent species data directly from char.csv and pike.csv
# This ensures we get ALL locations, including any that might be missing from df_current
char_df = pd.read_csv('char.csv')
pike_df = pd.read_csv('pike.csv')

# Standardize char.csv columns (column names are swapped: Y_coord=lat, X_coord=lon)
# We need: X_coord=longitude, Y_coord=latitude
char_df = char_df.rename(columns={'Y_coord': 'Y_coord_lat', 'X_coord': 'X_coord_lon'})
char_df['X_coord'] = char_df['X_coord_lon']  # X_coord = longitude
char_df['Y_coord'] = char_df['Y_coord_lat']  # Y_coord = latitude
char_df = char_df.drop(columns=['Y_coord_lat', 'X_coord_lon'])

# Standardize pike.csv columns (column names are swapped: Y_coord=lat, X_coord=lon)
pike_df = pike_df.rename(columns={'Y_coord': 'Y_coord_lat', 'X_coord': 'X_coord_lon'})
pike_df['X_coord'] = pike_df['X_coord_lon']  # X_coord = longitude
pike_df['Y_coord'] = pike_df['Y_coord_lat']  # Y_coord = latitude
pike_df = pike_df.drop(columns=['Y_coord_lat', 'X_coord_lon'])

# Add missing columns to match expected format
# char.csv has ABUNDANCE, pike.csv doesn't
if 'ABUNDANCE' not in pike_df.columns:
    pike_df['ABUNDANCE'] = None
if 'DATE' not in char_df.columns:
    char_df['DATE'] = None
if 'DATE' not in pike_df.columns:
    pike_df['DATE'] = None
if 'QTY' not in char_df.columns:
    char_df['QTY'] = None
if 'QTY' not in pike_df.columns:
    pike_df['QTY'] = None
if 'SIZE (inch)' not in char_df.columns:
    char_df['SIZE (inch)'] = None
if 'SIZE (inch)' not in pike_df.columns:
    pike_df['SIZE (inch)'] = None

# Combine char and pike data
permanent_data = pd.concat([char_df, pike_df], ignore_index=True)

# Ensure proper column order
column_order = ['COUNTY', 'DATE', 'WATER', 'TOWN', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord', 'ABUNDANCE']
permanent_data = permanent_data[column_order]

print(f"Permanent species data: {len(permanent_data)} rows")
print(f"  Arctic Char: {len(permanent_data[permanent_data['SPECIES']=='ARCTIC CHAR'])} rows")
print(f"  Northern Pike: {len(permanent_data[permanent_data['SPECIES']=='NORTHERN PIKE'])} rows")
print(f"Species: {sorted(permanent_data['SPECIES'].unique())}")
permanent_data.head()


Permanent species data: 44 rows
  Arctic Char: 14 rows
  Northern Pike: 29 rows
Species: ['ARCTIC CHAR', 'NORTHENR PIKE', 'NORTHERN PIKE']


,COUNTY,DATE,WATER,TOWN,SPECIES,QTY,SIZE (inch),X_coord,Y_coord,ABUNDANCE
0,Aroostook,None,BLACK P,T15R9 WELS,ARCTIC CHAR,None,None,-68.839522,46.974063,High
1,Aroostook,None,DEBOULLIE P,T15R9 WELS,ARCTIC CHAR,None,None,-68.855618,46.965112,Moderate
2,Aroostook,None,GARDNER P,T15R9 WELS,ARCTIC CHAR,None,None,-68.885971,46.960960,Moderate
3,Aroostook,None,PUSHINEER P,T15R9 WELS,ARCTIC CHAR,None,None,-68.841245,46.960786,Low
4,Franklin,None,LONG P,Township E,ARCTIC CHAR,None,None,-70.089434,45.626023,High


In [32]:
# Combine 2025 data with permanent species data
# Ensure both have same columns (add ABUNDANCE column for char data)
df_2025_final['ABUNDANCE'] = None

# Ensure column order matches
column_order = ['COUNTY', 'DATE', 'WATER', 'TOWN', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord', 'ABUNDANCE']
df_2025_final = df_2025_final[column_order]
permanent_data = permanent_data[column_order]

# Combine dataframes
df_updated_new = pd.concat([df_2025_final, permanent_data], ignore_index=True)

print(f"Combined dataset: {len(df_updated_new)} rows")
print(f"\nSpecies breakdown:")
print(df_updated_new['SPECIES'].value_counts().sort_index())
print(f"\nColumns: {df_updated_new.columns.tolist()}")
df_updated_new.head()


Combined dataset: 2251 rows

Species breakdown:
SPECIES
ARCTIC CHAR        14
BROOK TROUT      1526
BROWN TROUT       330
L.L. SALMON       200
LAKE TROUT         11
NORTHENR PIKE       1
NORTHERN PIKE      29
RAINBOW TROUT      80
SPLAKE             60
Name: count, dtype: int64

Columns: ['COUNTY', 'DATE', 'WATER', 'TOWN', 'SPECIES', 'QTY', 'SIZE (inch)', 'X_coord', 'Y_coord', 'ABUNDANCE']


/var/folders/pm/v1wywnps12sf4vmw4zxrydwr0000gn/T/ipykernel_64046/4010754209.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_updated_new = pd.concat([df_2025_final, permanent_data], ignore_index=True)


,COUNTY,DATE,WATER,TOWN,SPECIES,QTY,SIZE (inch),X_coord,Y_coord,ABUNDANCE
0,Androscoggin,2025-02-11,RANGE P (LOWER),Poland,BROOK TROUT,300,14,-70.369148,44.043731,None
1,Cumberland,2025-04-03,SONGO R,Casco,BROOK TROUT,100,16,-70.574503,43.930731,None
2,Cumberland,2025-04-03,JORDAN R,Raymond,BROOK TROUT,100,16,-70.462052,43.896887,None
3,Cumberland,2025-04-03,PRESUMPSCOT R,Windham,BROOK TROUT,200,16,-70.410724,43.717251,None
4,Cumberland,2025-04-07,COLLYER B,Gray,BROOK TROUT,100,10,-70.302713,43.905907,None


In [33]:
# Create coordinate reference file with all unique coordinates
# Columns: WATER, TOWN, COUNTY, X_coord, Y_coord
coord_reference = df_updated_new[
    df_updated_new['X_coord'].notna() & df_updated_new['Y_coord'].notna()
][['WATER', 'TOWN', 'COUNTY', 'X_coord', 'Y_coord']].drop_duplicates(
    subset=['WATER', 'TOWN', 'COUNTY']
).sort_values(['COUNTY', 'TOWN', 'WATER']).reset_index(drop=True)

print(f"Coordinate reference: {len(coord_reference)} unique locations")
print(f"\nColumns: {coord_reference.columns.tolist()}")
coord_reference.head(10)


Coordinate reference: 957 unique locations

Columns: ['WATER', 'TOWN', 'COUNTY', 'X_coord', 'Y_coord']


,WATER,TOWN,COUNTY,X_coord,Y_coord
0,ANDROSCOGGIN R (LITTLE),Auburn,Androscoggin,-70.239381,44.071433
1,AUBURN L,Auburn,Androscoggin,-70.253311,44.144799
2,BASIN B,Auburn,Androscoggin,-70.286303,44.184689
3,PETTINGILL PARK P,Auburn,Androscoggin,-70.234543,44.106918
4,ROYAL R,Auburn,Androscoggin,-70.260466,44.006769
5,TAYLOR P,Auburn,Androscoggin,-70.278494,44.109052
6,MEADOW B,Durham,Androscoggin,-70.069211,43.975001
7,NEWELL B,Durham,Androscoggin,-70.1009,43.98565
8,SABATTUS P (LITTLE),Greene,Androscoggin,-70.134896,44.210982
9,STETSON B,Lewiston,Androscoggin,-69.123939,44.838304


In [34]:
# Save updated data
print("Saving files...")

# Save df_updated.csv
df_updated_new.to_csv('df_updated.csv', index=False)
print(f"✅ Saved df_updated.csv with {len(df_updated_new)} rows")

# Save coordinate_reference.csv
coord_reference.to_csv('coordinate_reference.csv', index=False)
print(f"✅ Saved coordinate_reference.csv with {len(coord_reference)} unique locations")

print("\n" + "=" * 80)
print("UPDATE COMPLETE!")
print("=" * 80)
print(f"\nSummary:")
print(f"  Total rows in df_updated.csv: {len(df_updated_new)}")
print(f"  Unique locations with coordinates: {len(coord_reference)}")
print(f"  2025 stocked species rows: {len(df_2025_final)}")
print(f"  Permanent species rows: {len(permanent_data)}")


Saving files...
✅ Saved df_updated.csv with 2251 rows
✅ Saved coordinate_reference.csv with 957 unique locations

UPDATE COMPLETE!

Summary:
  Total rows in df_updated.csv: 2251
  Unique locations with coordinates: 957
  2025 stocked species rows: 2207
  Permanent species rows: 44
